In [1]:
# hard negative 만들기
from utils.tools import *
from utils.utils import load_jsonl, save_jsonl
from tqdm import tqdm
from transformers import AutoTokenizer
import argparse
from retrieval.sparse_retrieval.bm25_retrieval import BM25
from sklearn.model_selection import train_test_split
import json

In [2]:
parser = argparse.ArgumentParser()
# test name
parser.add_argument('--data_path', type=str, default='./data/wiki.json')
parser.add_argument('--test_data_path', type=str, default='./data/train.jsonl')
parser.add_argument('--include_title', type=str2bool, default=True)

_StoreAction(option_strings=['--include_title'], dest='include_title', nargs=None, const=None, default=True, type=<function str2bool at 0x7f6ad45aff70>, choices=None, help=None, metavar=None)

In [3]:
args,_ = parser.parse_known_args()

In [4]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('../models/klue_bert_tokenizer')

In [5]:
bm25 = BM25(args, tokenizer)
bm25.exec_embedding()

tokenize: 100%|█████████████████████| 1346460/1346460 [05:23<00:00, 4159.00it/s]


In [ ]:
# test_data = load_jsonl('./data/dev.jsonl')

# test_data score 기록

In [8]:
result = []
for i in tqdm(test_data):
    output = bm25.retrieve(i['question'],100)
    output = [i[0] for i in output]
    result.append(output)

100%|████████████████████████████████████| 4909/4909 [13:39:41<00:00, 10.02s/it]


In [17]:
# hit@1, 5, 20, 100
scores = []
for k in [1,5,20,100]:
    score_k = []
    for i,j in tqdm(zip(test_data,result)):
        score_k.append(i['positive_ctxs_ids'][0] in j[:k])
    scores.append(score_k)

4909it [00:00, 391717.34it/s]
4909it [00:00, 357406.63it/s]
4909it [00:00, 277016.94it/s]
4909it [00:00, 171128.50it/s]


In [18]:
for i in scores:
    print(sum(i)/len(i))

0.567936443267468
0.7751069464249338
0.8653493583214504
0.927072723568955


# hard negative attach

In [6]:
train_data = load_jsonl('./data/train.jsonl')

51237it [00:00, 152080.37it/s]


In [7]:
for i in tqdm(train_data):
    pos,neg = bm25.retrieve_without(i['question'], i['answer'], 100, 1)
    i['hard_negative_ctxs_ids'] = neg
    i['retrieved_ctxs_ids'] = pos

100%|█████████████████████████████████| 51237/51237 [141:23:15<00:00,  9.93s/it]


In [28]:
train_data, val_data = train_test_split(train_data, test_size = 0.1, random_state = 42, shuffle = True)

In [21]:
wiki = json.load(open(args.data_path))

In [29]:
for i in tqdm(train_data):
    i['negative_ctxs_ids'] = i['hard_negative_ctxs_ids']
    del i['hard_negative_ctxs_ids']
    i['negative_ctxs'] = [wiki[j] for j in i['negative_ctxs_ids']]
    i['sparse_retrieved_ctxs_ids'] = i['retrieved_ctxs_ids']
    del i['retrieved_ctxs_ids']

100%|█████████████████████████████████| 41501/41501 [00:00<00:00, 305810.26it/s]


In [30]:
for i in tqdm(val_data):
    i['negative_ctxs_ids'] = i['hard_negative_ctxs_ids']
    del i['hard_negative_ctxs_ids']
    i['negative_ctxs'] = [wiki[j] for j in i['negative_ctxs_ids']]
    i['sparse_retrieved_ctxs_ids'] = i['retrieved_ctxs_ids']
    del i['retrieved_ctxs_ids']

100%|███████████████████████████████████| 4612/4612 [00:00<00:00, 194561.98it/s]


In [5]:
train_data = load_jsonl('./data/train_data_attached.jsonl')
val_data = load_jsonl('./data/val_data_attached.jsonl')

41501it [00:05, 7785.65it/s] 
4612it [00:00, 5766.54it/s] 


In [9]:
val_data = load_jsonl('./data/dev.jsonl')

4909it [00:00, 71522.05it/s]


In [ ]:
for i in train_data:
    i['neagtive

In [34]:
save_jsonl('./data',train_data, 'train_data_attached')
save_jsonl('./data',val_data, 'val_data_attached')

# check_hard_negative

In [22]:
import json

In [24]:
wiki = json.load(open(args.data_path))

In [25]:
wiki['오우데나르데 전투_19']

{'title': '오우데나르데 전투',
 'context': '군대는 오우데나르데 근처의 임시 다리가 붕괴되는 바람에 강을 건널 수가 없었고, 도시의 돌다리를 이용하여 강을 건너야 했기 때문에 한시간정도 늦어지게 되었다. 말버러 공작은 계획에 착수하여 외젠 공작의 기병대에게 돌격을 명했다. 이 돌격은 부르고뉴 공작의 사령부에 행해졌다. 프랑스군의 본진을 지키던 기병대는 메종 드 로이(Maison du Roi)부대였는데, 이들은 외젠 공작의 기병대에 대하여'}

In [30]:
check = []
for i in tqdm(train_data):
    assert i['answer'] not in wiki[i['hard_negative_ctxs_ids'][0]]['context']

100%|█████████████████████████████████| 50724/50724 [00:00<00:00, 339804.40it/s]
